# Декоратор @cached

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [1]:
import functools
import collections
import time

In [2]:
class LruCache:
    def __init__(self, max_count):
        self.__cache = collections.OrderedDict()
        self.__cache_count = max_count

    def __getitem__(self, key):
        return self.__cache.get(key)

    def __setitem__(self, key, value):
        self.__cache[key] = value
        if self.__cache_count < len(self.__cache):
            self.__cache.popitem(last=False)
    
    def __contains__(self, key):
        return key in self.__cache

#### Реализуйте декоратор

In [3]:
def cached(max_count):
    def decorator(func):
        cache = LruCache(max_count)
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = tuple(list(args) + list(tuple(kwargs.items())))
            if key not in cache:
                cache[key] = func(*args, **kwargs)
            return cache[key]
        
        return wrapper
    
    return decorator

#### Проверьте использование декоратора

Сравним время работы функции подсчета факториала числа с использованием декоратора `@cached` и без его использования.

In [4]:
@cached(20)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

In [5]:
cached_start = time.time()

for i in range(1000):
    fact(i + 1)

cached_finish = time.time()
print("Running time with @cached: {}".format(cached_finish - cached_start))

not_cached_start = time.time()

for i in range(1000):
    fact1(i + 1)

not_cached_finish = time.time()
print("Running time without @cached: {}".format(not_cached_finish - not_cached_start))

Running time with @cached: 0.0036826133728027344
Running time without @cached: 0.14880943298339844


#### Сравните свою реализацию с lru_cache из functools

In [181]:
not_cached_start = time.time()

for i in range(1000):
    fact1(i + 1)

not_cached_finish = time.time()
print(not_cached_finish - not_cached_start)

0.16572046279907227


In [6]:
from functools import lru_cache

@lru_cache(maxsize = 2)
def fact3(n):
    if n < 2:
        return 1
    return fact3(n-1) * n

In [7]:
cached_start = time.time()

for i in range(1000):
    fact(i + 1)

cached_finish = time.time()
print("Running time with @cached: {}".format(cached_finish - cached_start))

lru_cache_start = time.time()

for i in range(1000):
    fact3(i + 1)

lru_cache_finish = time.time()
print("Running time with @lru_cache: {}".format(lru_cache_finish - lru_cache_start))

Running time with @cached: 0.005354642868041992
Running time with @lru_cache: 0.0013930797576904297
